< desteklenen model gemini-1.5-flash>


In [3]:
# Requirementsları indir
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langgraph-core (from versions: none)
ERROR: No matching distribution found for langgraph-core


In [2]:
%pip install -U langchain-community

#Kütüphane importları
from dotenv import load_dotenv     # .env dosyasını yüklemek için API key
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings # Google LLM kullanmak için
import requests # HTTP istekleri yapmak için
import time # Zamanlama işlemleri için - 1 saniyelik bekleme 
import os # Çevresel değişkenleri yönetmek için
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached aiohttp-3.12.15-cp311-cp311-win_amd64.whl.metadata (7.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached frozenlist-1.7.0-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached multidict-6.6.3-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached propcache-0.3.2-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp311-cp311-win_amd64.whl.metadata (76 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata

c:\Users\Hp\.conda\envs\AI_Agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

load_dotenv() # env daki keyi os de cek ve kullan
# Örnek kullanım
LLM = GoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1,
)

response = LLM.invoke("What is the capital of France?")  # Example usage
print(response)  # Output the response from the model

The capital of France is Paris.


In [4]:
# Dinlenecek endPoint Adresleri
ENDPOINT_ATTENTION = "http://127.0.0.1:8001/attention"
ENDPOINT_SCRIPT = "http://localhost:5000/get_texts"
ENDPOINT_KYBRD_MOUSE_INTERRUPT = "http://localhost:5000/api/status"


In [ ]:
# Veri çekme ve işleme döngüsü
while True:
    try:
        response = requests.get(ENDPOINT_ATTENTION)
        data = response.json()
    except Exception as e:
        print(f"[HATA] ENDPOINT_ATTENTION verisi alınamadı: {e}")
        data = {}

    try:
        keyboard_response = requests.get(ENDPOINT_KYBRD_MOUSE_INTERRUPT)
        keyboard_data = keyboard_response.json()
    except Exception as e:
        print(f"[HATA] ENDPOINT_KYBRD_MOUSE_INTERRUPT verisi alınamadı: {e}")
        keyboard_data = {}

    try:
        script_response = requests.get(ENDPOINT_SCRIPT)
        script_data = script_response.json()
    except Exception as e:
        print(f"[HATA] ENDPOINT_SCRIPT verisi alınamadı: {e}")
        script_data = {}

    time.sleep(1)


In [ ]:
# 🧬 Embedder ve ChromaDB yapılandırması (ilk kullanımda klasör oluşturur)
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectordb = Chroma(persist_directory="student_memory", embedding_function=embedding)

# 🔁 RetrievalQA ile geçmiş öğrenci verilerini LLM'e bağla
qa_chain = RetrievalQA.from_chain_type(
    llm=LLM,
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff"
)


# 🧩 Son gelen dikkat verilerini ChromaDB'ye kaydet (isteğe bağlı - sadece örnek amaçlı)
metadata = {
    "attention": data.get("attention", 0),
    "screen": data.get("head_looking_at_screen", False),
    "eye_left": data.get("left_eye_open", False),
    "eye_right": data.get("right_eye_open", False),
    "att_1min": data.get("attention_1min_avg", 0),
    "att_5min": data.get("attention_5min_avg", 0),
    "att_20min": data.get("attention_20min_avg", 0),
    "att_total": data.get("attention_total_avg", 0),
    "keyboard_activity": keyboard_data.get("keyboard_activity", False),
    "mouse_activity": keyboard_data.get("mouse_activity", False),
    "tab_changed": keyboard_data.get("tab_changed", False),
    "interaction_status": keyboard_data.get("status", "no_status"),
    "focus_score": script_data.get("focus_score", 0) #script_data'dan alınan script benzerlik değeri
}

query = f"""
🎓 SEN, yalnızca dikkat düzeyini değil, öğrencinin **öğrenme stilini** (VAK modeli: görsel, işitsel, kinestetik) analiz edebilen ileri düzey bir eğitim bilimci ve kişisel koçsun.

Aşağıda, öğrencinin dikkat, göz takibi, ekran izleme, focus score, klavye/mouse aktivitesi gibi çoklu verileri yer alıyor. Görevin:

---

## 🎯 1. Dikkat Verisi Değerlendirmesi

* Anlık dikkat düzeyi (attention), geçmiş ortalamalar (1dk, 5dk, 20dk)
* Ekrana bakma durumu ve göz açıklıkları
* Focus score (senaryo içeriğine odaklılık)

Bunları değerlendirerek öğrencinin **anlık dikkat seviyesini** yorumla ve gerekirse çözüm üretici öneriler sun.

---

## 🧠 2. Öğrenme Tarzı (VAK) Tahmini

Verilere dayanarak öğrencinin baskın öğrenme tarzını (görsel, işitsel, kinestetik) **tahmin et**. Eğer karışık bir profil varsa bu kombinasyonu açıkla.

- Göz takibi, ekrana odak, yüksek attention → görsel
- Focus score yüksek, sesli konuşma sırasında dikkatli → işitsel
- Klavye/mouse etkileşimi, sekme değişimi → kinestetik

---

## 🛠️ 3. Öğrenciye Özel Uygulama Önerileri

Her tarza uygun aşağıdaki örneklerden yararlanarak öneriler ver:

| Öğrenme Tarzı | Öneriler |
|---------------|----------|
| Görsel | Diyagram, video, grafik, kavram haritası |
| İşitsel | Podcast, sesli anlatım, tekrar ses kayıtları |
| Kinestetik | Uygulamalı etkinlikler, mini projeler, yaz‑çiz‑hareket |

---

## ✨ 4. Geri Bildirim Metni

Destekleyici, motive edici, koçvari bir geri bildirim oluştur.

- Dikkat yüksekse: Takdir et, güçlü yönleri vurgula
- Dikkat düşükse: Cesaretlendir, çözüm öner
- Her zaman pozitif yaklaşım kullan

---

## 🔢 Dikkat & Etkileşim Verileri

- Dikkat skoru: {data.get('attention',)}
- 1dk ort.: {data.get('attention_1min_avg')}
- 5dk ort.: {data.get('attention_5min_avg')}
- 20dk ort.: {data.get('attention_20min_avg')}
- Genel ort.: {data.get('attention_total_avg')}
- Ekrana bakıyor mu: {data.get('head_looking_at_screen')}
- Göz durumu: Sol={data.get('left_eye_open', False)}, Sağ={data.get('right_eye_open')}
- Focus Score (senaryo içeriğine odak): {script_data.get('focus_score')}
- Klavye: {keyboard_data.get('keyboard_activity')}
- Mouse: {keyboard_data.get('mouse_activity')}
- Sekme değişti mi: {keyboard_data.get('tab_changed')}
- Etkileşim durumu kodu: {keyboard_data.get('status')}

---

Lütfen önce veri analizi yap, ardından öğrenme tarzını tahmin et, sonra önerilerde bulun ve en son bir paragraflık destekleyici koçluk geri bildirimi oluştur.
"""

vectordb.add_texts([query], metadatas=[metadata])


coach_feedback = qa_chain.run(query)

print("\n🎓 Koç Tavrı ile Kişisel Geri Bildirim:")
print(coach_feedback)


In [ ]:
#Öğrenci Fonksiyonları
# 1 - Dikkatin Dağıldı Uyarısı (Görsel & Sesli)
# ✅ DİKKAT DAĞILDI UYARI MODÜLÜ
def dikkat_uyarisi_ver(data: dict):
    try:
        dikkat_dagit = False

        # 👀 Ekrana bakmıyorsa veya dikkat seviyesi çok düşükse
        if not data["head_looking_at_screen"] or data["attention"] < 0.5:
            dikkat_dagit = True

        if dikkat_dagit:
            print("⚠️ Dikkatin dağıldı! Lütfen odaklan.")
            
            # 🔊 Sesli uyarı (Windows için)
            try:
                import winsound
                winsound.Beep(1000, 400)
            except:
                pass  # MacOS/Linux için farklı çözümler eklenebilir

            # 🎨 Terminal görsel simülasyonu (UI'ye entegre edilebilir)
            print("\033[91m[Çerçeve RENGİ: KIRMIZI - ODAK YOK]\033[0m")

    except Exception as e:
        print(f"🚨 Dikkat modülü hatası: {e}")
        
        
# 2 - Yorgunluk Algılama ve Mola Önerisi
# ✅ CROMADB TABANLI YORGUNLUK ALGILAMA MODÜLÜ
def mola_onerisi_chromadb(vectordb):
    try:
        # Chroma'dan son 10 kaydı al
        recent_docs = vectordb.similarity_search("attention", k=10)

        # Geçerli kayıtlar varsa, dikkat_20min skorlarını topla
        attention_20mins = []
        for doc in recent_docs:
            att = doc.metadata.get("att_20min")
            if isinstance(att, (int, float)):
                attention_20mins.append(att)
        
        # Ortalama hesapla
        if len(attention_20mins) >= 3:  # En az 3 veri noktası şartı
            avg_att_20min = sum(attention_20mins) / len(attention_20mins)

            if avg_att_20min < 0.6:
                print("😴 Son 20 dakikada dikkat seviyen ortalama %60'ın altında.")
                print("💡 Bu odak kaybı, zihinsel yorgunluğa işaret ediyor olabilir.")
                print("⏸️ Kısa bir mola vermeyi dene! Bir yürüyüş veya esneme iyi gelebilir.")

                try:
                    import winsound
                    winsound.Beep(800, 500)
                except:
                    pass

                print("\033[95m[Mola Tavsiyesi: Chroma geçmiş analizine dayalı düşük dikkat algılandı.]\033[0m")

    except Exception as e:
        print(f"🚨 Chroma yorgunluk modülü hatası: {e}")



# 3 - LLM Destekli Öğrenme Periyodu Önerisi
# ✅ LLM DESTEKLİ, TEKNİK SINIRLAMASI OLMAYAN ÖĞRENME PERİYODU ÖNERİSİ
def periyot_onerisi_llm_ileri(vectordb, qa_chain):
    try:
        recent_docs = vectordb.similarity_search("attention", k=15)
        attention_values = []
        focus_scores = []
        metadata_list = []

        for doc in recent_docs:
            att = doc.metadata.get("attention")
            focus = doc.metadata.get("focus_score", 0)

            if isinstance(att, (float, int)):
                attention_values.append(att)
            if isinstance(focus, (float, int)):
                focus_scores.append(focus)

            metadata_list.append(doc.metadata)

        if len(attention_values) < 5:
            print("📊 Öğrenciye özel öneri için yeterli veri yok.")
            return

        avg_attention = sum(attention_values) / len(attention_values)
        avg_focus_score = sum(focus_scores) / len(focus_scores) if focus_scores else 0

        attention_summary = "\n".join([
            f"- Dikkat: {round(m.get('attention', 0), 2)}, 20dk Ort: {round(m.get('att_20min', 0), 2)}, 🎙️ Focus Score: {round(m.get('focus_score', 0), 2)}, Klavye: {m.get('keyboard_activity')}, Mouse: {m.get('mouse_activity')}, Sekme: {m.get('tab_changed')}"
            for m in metadata_list
        ])

        prompt = f"""
📚 SEN, deneyimli bir eğitim bilimci ve koçsun. Aşağıdaki dikkat geçmişine sahip bir öğrenciyi analiz ederek ona en uygun **çalışma periyodu ve öğrenme tekniğini** önereceksin.

🔍 Sadece klasik tekniklerle sınırlı kalma! Gerekirse aşağıdaki gibi alternatif tekniklerden birini ya da birkaçını birleştir:
- 52/17 Tekniği
- Ultradian Ritim
- Time Blocking
- Active Recall
- Feynman Tekniği
- Zihin Haritası ile Bölümlendirme
- Pomodoro
- Deep Work
- Interleaving / Spaced Repetition
- Görsel-Akış Haritası Tekniği
- Motion-Based Learning (kinestetikler için)
- Mikro Görev Parçalama vs.

🎯 Görevlerin:
1. Öğrencinin dikkat alışkanlıklarını ve davranış örüntülerini analiz et.
2. En uygun 1 veya 2 teknik öner. Sebepleriyle açıkla.
3. Nasıl uygulanacağını aşamalı olarak anlat.
4. Günlük takip planı öner.

🧠 Veriler:
{attention_summary}

📊 Ortalama dikkat skoru: {round(avg_attention, 2)}
🎙️ Odaklı Konuşma Skoru (focus_score): {round(avg_focus_score, 2)}

💡 Ek Not: Focus Score, öğrencinin öğrenme sırasında konuşma senaryosu içeriğine olan odaklanma düzeyidir. Yüksekse, sesli anlatım ve sözel içeriklerin etkili olabileceği bir öğrenme stiline işaret edebilir.
"""

        feedback = qa_chain.run(prompt)

        print("\n🧠 LLM Destekli Kişisel Öğrenme Stratejisi Önerisi:")
        print(feedback)

    except Exception as e:
        print(f"🚨 Gelişmiş LLM öneri modülü hatası: {e}")


In [ ]:
# LLM’in kullanabileceği araçları tanımlıyoruz
tools = [
    Tool(
        name="DikkatUyarisi",
        func=dikkat_uyarisi_ver,
        description="Öğrencinin dikkat seviyesi düşükse uyarı verir. (attention < 0.5)"
    ),
    Tool(
        name="MolaOnerisi",
        func=lambda x: mola_onerisi_chromadb(vectordb),
        description="Son 20 dakikadaki dikkat ortalaması düşükse mola önerir. (att_20min < 0.6)"
    ),
    Tool(
        name="OgrenmePeriyoduOnerisi",
        func=lambda x: periyot_onerisi_llm_ileri(vectordb, qa_chain),
        description="Öğrenci dikkat geçmişine göre en uygun öğrenme stratejisini önerir."
    )
]

In [ ]:
# Agent oluşturma
# LLM + Tool’ları bir araya getirerek agent oluşturuyoruz
agent = initialize_agent(
    tools=tools,
    llm=LLM,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)



In [ ]:
# Agent Prompt - Görevi

agent_prompt = """
Aşağıda öğrencinin dikkat verileri yer almakta.
Sen deneyimli bir eğitim koçusun. Bu verileri analiz et, duruma göre aşağıdaki görevleri sırayla uygula:

1. Dikkat düşükse uyarı ver.
2. Son 20dk dikkat ortalaması düşükse mola öner.
3. Öğrencinin dikkat geçmişine göre çalışma periyodu ve tekniği öner.

Gerekli görevleri sırasıyla kendi kararınla seç ve uygula.
"""

agent.run(agent_prompt)

In [ ]:
# 1 - Dikkatin Dağıldı Uyarısı (Görsel & Sesli) TEST
dummy_data = {
    "attention": 0.42,
    "head_looking_at_screen": False,
    "left_eye_open": True,
    "right_eye_open": True,
    "attention_1min_avg": 0.51,
    "attention_5min_avg": 0.54,
    "attention_20min_avg": 0.50,
    "attention_total_avg": 0.52
}

# dikkat_uyarisi_ver(dummy_data)


In [ ]:
# 2 - Yorgunluk Algılama ve Mola Önerisi TEST

# 🧪 TEST (Senaryo: son 20dk ortalaması çok düşük)
# 🧪 Vectordb üstünden test etmek için çağır
# mola_onerisi_chromadb(vectordb)



In [ ]:
# 3 - LLM Destekli Öğrenme Periyodu Önerisi TEST

#periyot_onerisi_llm_ileri(vectordb, qa_chain)
